## On the Mahler-Popken (MP) complexity 

The Mahler-Popken complexity of an integer $n, mp(n)$, is the minimal number of 1s required to build $n$ using only addition and multiplication operators.

The basic insight is to realise that the solution relies on the fundamental theorem of arithmetic, that is that every integer has a unique prime factorisation. Suppose $n=p_1^{a_1}p_2^{a_2}..p_k^{a_k}$, where without loss of generality we assume that $p_i < p_j$ for $i <j$. Then the calculation of $mp$ can proceed in one of two directions, depending on whether $n$ is prime or composite. 

In $n$ is prime then $mp(n)$ will be $mp(a)+mp(b)$ derived from a partition of n into two parts: $a,b$ such that $a+b=n$. If $n$ is composite then it may also be $mp(a)*mp(b)$ where  $a,b$ is a complementary pair of factors, that is $a*b=n$.

More formally, if n prime $> 2$ then the minimum number will be in the sequence $\{mp(n-1)+mp(1), mp(n-2)+mp(2),\cdots, mp(n-d)+mp(d)\}$, where $d = n \;\texttt{`div`}\; 2$ otherwise it will take that form or $mp(n \; \texttt{`div`} \; m)+mp(m)$ where $m$ is some divisor of $n$. 

We will need some helper functions to help us work with divisors and their complementary pairs. These are taken from oeis.org -- the On-line Encyclopedia of Integer Sequences. OEIS is an early port of call for mathematicians who are looking to explore a research theme. By entering a sequence into the database they will see where else it has appeared, and thus be directed to another, possibly connected, area of mathematics (Terence Tao. AI and mathematics. IMO, 2024. https:// www.youtube.com/watch?v=e049IoFBnLA.)  

The hints we supplied to the problem in Week 24 suggested that you first work with pen and pencil to derive a possible sequence of values of mp(n) where n is [1..15] (say), and then enter this as a comma separated list of integers into OEIS to check if your working is correct.

## Functions to generate and access a  triangular table of the divisors of n

The figure shows the dependencies between the functions <img src="images/a1779101IaBe102213A1FFragsOEIS.png" width="400"/>

In [1]:
a027750 n k = a027750_row n !! (k-1)
a027750_row n = filter ((== 0) . mod n) [1..n]
a027750_tabf = map a027750_row [1..]

The `triangular table' is a list of lists. It begins

1;

1, 2;

1, 3;

1, 2, 4;

1, 5;

1, 2, 3, 6;

1, 7;

1, 2, 4, 8;

1, 3, 9;

1, 2, 5, 10;

1, 11;

1, 2, 3, 4, 6, 12;

In [2]:
-- function call to return row 6
a027750_row 6 

[1,2,3,6]

In [3]:
-- function call to return second entry in row 6
a027750 6 2

2

In [4]:
-- function to return a list of the first 12 entries in an infinite stream of divisor lists
take 12 a027750_tabf

[[1],[1,2],[1,3],[1,2,4],[1,5],[1,2,3,6],[1,7],[1,2,4,8],[1,3,9],[1,2,5,10],[1,11],[1,2,3,4,6,12]]

## Functions to generate and access triangular table of divisors of n <= sqrt n
If we define a divisor d|n to be `inferior' if d <= n/d, then inferior divisors are listed by this sequence

In [5]:
a161906 n k = a161906_tabf !! (n-1) !! (k-1)
a161906_row n = a161906_tabf !! (n-1)
a161906_tabf = zipWith (\m ds -> takeWhile ((<= m) . (^ 2)) ds)
                       [1..] a027750_tabf

In [6]:
a161906_row 6 -- function call to return row 6
a161906 6 2 -- function call to return second entry in row 6
-- function to return a list of the first 12 entries in an infinite stream of inferior divisor lists
take 12 a161906_tabf
-- Note that primes have row = [1]

[1,2]

2

[[1],[1],[1],[1,2],[1],[1,2],[1],[1,2],[1,3],[1,2],[1],[1,2,3]]

 ## Functions to generate and access a triangular table of `superior' divisors d
 
 A divisor is `superior' if d >= n/d

In [7]:
a161908 n k = a161908_tabf !! (n-1) !! (k-1)
a161908_row n = a161908_tabf !! (n-1)
a161908_tabf = zipWith
               (\x ds -> reverse $ map (div x) ds) [1..] a161906_tabf

In [8]:
a161908_row 6 -- function to return row 6
a161908 6 2 -- function to return second entry in row 6
-- function to return a list of the first 12 entries in an infinite stream of superior divisor lists
take 12 a161908_tabf

[3,6]

6

[[1],[2],[3],[2,4],[5],[3,6],[7],[4,8],[3,9],[5,10],[11],[4,6,12]]

In [9]:
map (div 1) [1]

[1]

In [10]:
map (div 2) [1]

[2]

In [11]:
map (div 12) [1,2,3]

[12,6,4]

## Mahler-Popken complexity of n: minimal number of 1's required to build n with + and *

The algorithm is based on https://oeis.org/A005245. 

The following cells test the sub-expressions in A005245. 

Note the use of the library functions `minimum` (returns the smallest element in a list) and `zipWith` (Rather than tupling the elements, the elements are combined using the function passed as the first argument).

The function a005245 is an infinite list of which the n'th element is $mp(n)$. 

There are two cases in the logic which we examine with test1 and test2.

In [32]:
import Data.List (genericIndex)
a005245 n = a005245_list `genericIndex` (n-1)
a005245_list = 1 : f 2 [1] where
   f x ys = y : f (x + 1) (y : ys) where
     y = minimum $ 
         zipWith (+) (take (x `div` 2) ys) (reverse ys) ++
         zipWith (+) (map a005245 $ tail $ a161906_row x)
                     (map a005245 $ reverse $ init $ a161908_row x)

Line 8: Use tail
Found:
reverse $ init $ a161908_row x
Why not:
tail (reverse (a161908_row x))

In [33]:
-- test with hand calculation [1,2,3,4,5,5,6,6,6,7,8,7]
take 12 a005245_list

[1,2,3,4,5,5,6,6,6,7,8,7]

## Initial Step

Start with the deepest nested expression on the last line with `x = 2` and `ys = [1]` Use `it` to refer to the result returned from the last cell.

In [34]:
a161908_row 2

[2]

In [ ]:
init it

[]

In [ ]:
reverse it

[]

In [ ]:
map a005245 it

[]

Do the same work on first argument of zipWith (+) on the penultimate line of cell 12

In [ ]:
map a005245 $ tail $ a161906_row 2

[]

Evaluate the zipWith on the penultimate line of cell 12

In [ ]:
zipWith (+) [][]

[]

Evalute `(take (x `div` 2) ys) (reverse ys)`, the  third line from the bottom of cell 12 with the same parameters, that is `x = 2`, `ys =[1]`

x `div` 2 = 1

take 1 [1] ~~~> 1

reverse ys ~~~> [1]

In [ ]:
take 1 [1]

[1]

In [ ]:
zipWith (+) [1] [1]

[2]

Evaluate `y = minimum $ [2] + []`

In [ ]:
minimum $ [2] ++ []

Line 1: Evaluate
Found:
[2] ++ []
Why not:
[2]Line 1: Use :
Found:
[2] ++ []
Why not:
2 : []

2

In [ ]:
a005245_list `genericIndex` 1

2

## Subsequent steps

In [ ]:
test1 x ys = minimum $
         zipWith (+) (take (x `div` 2) ys) (reverse ys) ++
         zipWith (+) (map a005245 $ tail $ a161906_row x)
                      (map a005245 $ reverse $ init $ a161908_row x)

Line 4: Use tail
Found:
reverse $ init $ a161908_row x
Why not:
tail (reverse (a161908_row x))

In [ ]:
test1 3 [1,2]

3

In [ ]:
test1 4 [1,2,3]

4

In [ ]:
test1 5 [1,2,3,4]

5

In [ ]:
test1 6 [1,2,3,4,5]

5

In [ ]:
test1 7 [1,2,3,4,5,5]

6

In [ ]:
-- this test returns [] just in the case of primes otherwise 
-- it returns a list of sums of MP(a) + MP(b) for all a,b such that a<b and a*b=n
test2 x = zipWith (+) (map a005245 $ tail $ a161906_row x)
                      (map a005245 $ reverse $ init $ a161908_row x)

Line 4: Use tail
Found:
reverse $ init $ a161908_row x
Why not:
tail (reverse (a161908_row x))

In [ ]:
map test2 [1..10]

[[],[],[],[4],[],[5],[],[6],[6],[7]]

In [ ]:
-- this test returns the value of MP(n) for n prime by adding 1 to the value of MP(n-1)
test3 x ys = minimum (zipWith (+) (take (x `div` 2) ys) (reverse ys))


In [ ]:
test3 7 [1,2,3,4,5,5]

6

Copyright (2024) Sociality Mathematics CIC, licence CC BY-NC-ND Attribution-NonCommercial-NoDerivs   https://creativecommons.org/licenses/